In [ ]:
# %run DOVSOutages.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Plot_Box_sns
import GrubbsTest

In [ ]:
df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\20240407_20240413\AllOPCOs\DRFResults\DOVSRepeats_detailed.pkl')

In [ ]:
df.index.get_level_values(1).value_counts()

In [ ]:
df[df.index.get_level_values(0)=='960038955']

In [ ]:
df[df.index.get_level_values(1).str.contains('_1')]

In [ ]:
conn_outages = Utilities.get_utldb01p_oracle_connection()
conn_aws = Utilities.get_athena_prod_aws_connection()

In [ ]:
my_meter_premise_info = pd.read_csv(r'C:\Users\s346557\Documents\my_meter_premise_info.csv')
my_meter_premise_info = Utilities_df.remove_prepend_from_columns_in_df(my_meter_premise_info)
my_meter_premise_info_dict = my_meter_premise_info.squeeze().to_dict()

In [ ]:
# mp_where_str = ''
# for i, (col_name,val) in enumerate(my_meter_premise_info_dict.items()):
#     str_to_append = f" AND {col_name}='{val}'"
#     if i==0:
#         str_to_append = str_to_append[5:]
#     mp_where_str += str_to_append
    
mp_where_str = "aep_premise_nb = '{}' AND serialnumber = '{}'".format(
    my_meter_premise_info_dict['prem_nb'], 
    my_meter_premise_info_dict['mfr_devc_ser_nbr'])

In [ ]:
#my_meter_premise_info_dict['prem_nb']

# -----------------------------------------------------
# Outages
# -----------------------------------------------------

# -----------------------------------------------------
# Usage
# -----------------------------------------------------

In [ ]:
cols_of_interest_met_prem = [
    'mfr_devc_ser_nbr',
    'mtr_stat_cd_desc',
    'first_in_srvc_dt',
    'phys_inst_dt',
    'inst_tod_cd',
    'inst_ts',
    'last_fld_test_date',
    'longitude',
    'latitude',
    'state_cd', 
    'prem_nb',
    'prem_stat_cd',
    'frst_turn_on_dt',
    'hsng_ctgy_cd',
    'srvc_pole_nb',
    'trsf_pole_nb',
    'curr_bill_acct_id',
    'curr_bill_acct_nb',
    'annual_kwh',
    'annual_max_dmnd',
    'latitude_nb',
    'longitude_nb',
    'station_nb', 
    'xfmr_nb'
]

cols_of_interest_usage = [
    'serialnumber', 
    'starttimeperiod', 
    'endtimeperiod', 
    'aep_endtime_utc', 
    'timezoneoffset', 
    'aep_derived_uom', 
    'aep_srvc_qlty_idntfr', 
    'value', 
    'aep_opco', 
    'aep_usage_dt']

In [ ]:
date_range = ['2020-01-01', '2020-02-01']

In [ ]:
mp_sql = SQLQuery(sql_select = SQLSelect(cols_of_interest_met_prem), 
                  sql_from = SQLFrom('default', 'meter_premise'), 
                  sql_where = SQLWhere([dict(field_desc='prem_nb ', comparison_operator='=', value='{}', needs_quotes=True), 
                                        dict(field_desc='mfr_devc_ser_nbr  ', comparison_operator='=', value='{}', needs_quotes=True)], 
                                       idxs=None, run_check=True) 
                 )
#-------------------------------------
usg_sql_where = SQLWhere([dict(field_desc='aep_opco', comparison_operator='=', value='oh', needs_quotes=True), 
                          dict(field_desc='aep_usage_dt', comparison_operator='BETWEEN', value=['{}','{}'], needs_quotes=True), 
                          dict(field_desc='aep_derived_uom', comparison_operator='=', value='VOLT', needs_quotes=True), 
                          dict(field_desc='aep_srvc_qlty_idntfr', comparison_operator='=', value='AVG', needs_quotes=True), 
                          dict(field_desc='aep_derived_uom', comparison_operator='=', value='KWH', needs_quotes=True)], 
                         idxs=None, run_check=True)
usg_sql_where.combine_where_elements([2,3], 'AND', close_gaps_in_keys=False)
usg_sql_where.combine_where_elements([2,4], 'OR', close_gaps_in_keys=True)

usg_sql = SQLQuery(sql_select = SQLSelect(cols_of_interest_usage), 
                   sql_from = SQLFrom('usage_nonvee', 'reading_ivl_nonvee'), 
                   sql_where = usg_sql_where
                 )
#-------------------------------------
sql_select = SQLSelect(cols_of_interest_met_prem, global_table_alias_prefix='MP', idxs=None, run_check=True)
sql_select.add_select_elements(cols_of_interest_usage, global_table_alias_prefix='U', idxs=None, run_check=True)

sql_join = SQLJoin(join_type='INNER', join_table='', 
                        join_table_alias='U', orig_table_alias='MP', list_of_columns_to_join=[['mfr_devc_ser_nbr', 'serialnumber']])

sql_partial = SQLQuery(sql_select = sql_select, 
                            sql_from = SQLFrom('MP'), 
                            sql_where = None, 
                            sql_join_coll = sql_join)

sql_partial_stmnt = sql_partial.get_sql_statement()
#-------------------------------------
mp_sql_stmnt = mp_sql.get_sql_statement(insert_n_tabs_to_each_line=1).format(my_meter_premise_info_dict['prem_nb'], 
                                                                             my_meter_premise_info_dict['mfr_devc_ser_nbr'])
usg_sql_stmnt = usg_sql.get_sql_statement(insert_n_tabs_to_each_line=1).format(date_range[0], date_range[1])

sql_full_stmnt = f"""
WITH MP
AS (
{mp_sql_stmnt}
), 
U AS (
{usg_sql_stmnt}
)

{sql_partial_stmnt}
"""
print(sql_full_stmnt)

In [ ]:
df = pd.read_sql(sql_full_stmnt, conn_aws)
df = Utilities_df.remove_table_aliases(df)
#-----
df_vlt = df[df['aep_derived_uom']=='VOLT'].copy()
df_kwh = df[df['aep_derived_uom']=='KWH'].copy()

In [ ]:
df_vlt['starttimeperiod_tz_aware'] = pd.to_datetime(df_vlt['starttimeperiod'])
df_vlt['starttimeperiod_local'] = df_vlt['starttimeperiod_tz_aware'].dt.tz_localize(None)
df_vlt['starttimeperiod_utc'] = df_vlt['starttimeperiod_tz_aware'].dt.tz_convert(None)
df_vlt = df_vlt.set_index('starttimeperiod_local', drop=False).sort_index()

# df_vlt = df_vlt.set_index('starttimeperiod', drop=False)
# df_vlt.index = pd.to_datetime(df_vlt.index)
# df_vlt = df_vlt.sort_index()

In [ ]:
df_kwh['starttimeperiod_tz_aware'] = pd.to_datetime(df_kwh['starttimeperiod'])
df_kwh['starttimeperiod_local'] = df_kwh['starttimeperiod_tz_aware'].dt.tz_localize(None)
df_kwh['starttimeperiod_utc'] = df_kwh['starttimeperiod_tz_aware'].dt.tz_convert(None)
df_kwh = df_kwh.set_index('starttimeperiod_local', drop=False).sort_index()

In [ ]:
df_vlt['value'].plot()

In [ ]:
ax = df_vlt['2020-01-01T00:00:00':'2020-01-02T00:00:00']['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = df_vlt['2020-01-01T00:00:00':'2020-01-08T00:00:00']['value'].plot()
ax.tick_params(axis='x', labelrotation=45);

In [ ]:
ax = df_vlt['2020-01-01T00:00:00':'2020-01-08T00:00:00']['value'].plot()

ax.xaxis.set_major_locator(dates.HourLocator(interval=4))
ax.xaxis.set_major_formatter(dates.DateFormatter('%H'))

# ax.xaxis.set_minor_locator(dates.DayLocator())
# ax.xaxis.set_minor_formatter(dates.DateFormatter('\n\n%d %b'))
# ax.tick_params(which='minor', length=0)


In [ ]:
df_vlt.resample('H').mean()

In [ ]:
df_vlt.resample('H').mean().plot()

In [ ]:
df_vlt.resample('4H').mean().plot()

In [ ]:
tmp_df = df_vlt['2020-01-01T00:00:00':'2020-01-08T00:00:00'].copy()

print(pd.infer_freq(tmp_df.index))
tmp_df.index.freq = '15T'

ax = tmp_df['value'].plot(figsize=(12,5))
tmp_df.rolling(window=4).mean()['value'].plot()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

In [ ]:
tmp_df

In [ ]:
ax = tmp_df['value'].plot(figsize=(12,5))
result = seasonal_decompose(tmp_df['value'], model='additive', period=96)  # model='mul' also works 
result.plot();

In [ ]:
span = 7
alpha = 2/(span+1)

tmp_df['SES12'] = SimpleExpSmoothing(tmp_df['value']).fit(smoothing_level=alpha,optimized=False).fittedvalues.shift(-1)

In [ ]:
tmp_df[['value', 'SES12']].plot(figsize=(12,5))

In [ ]:
df_vlt

In [ ]:
n_train = round(0.8*df_vlt.shape[0])
n_test = df_vlt.shape[0]-n_train

train_data = df_vlt[:n_train]
test_data = df_vlt[n_train:]

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
fitted_model = ExponentialSmoothing(train_data['value'],trend='add',seasonal='add',seasonal_periods=7*24*4).fit()

In [ ]:
test_predictions = fitted_model.forecast(test_data.shape[0])

In [ ]:
train_data['value'].plot(legend=True,label='TRAIN')
test_data['value'].plot(legend=True,label='TEST',figsize=(12,8))
test_predictions.plot(legend=True,label='PREDICTION')

In [ ]:
train_data['value'].plot(legend=True,label='TRAIN')
test_data['value'].plot(legend=True,label='TEST',figsize=(12,8))
test_predictions.plot(legend=True,label='PREDICTION', xlim=['2020-01-26 14:30:00', '2020-02-01 23:45:00	'])